# OBJETIVO

O objetivo deste projeto é realizar web scraping do site do IBGE para coletar dados dos estados brasileiros e organizar essas informações em um dataframe do Pandas. Após essa estruturação, será feita a limpeza e o tratamento dos dados, removendo caracteres indesejados e ajustando formatos, garantindo a consistência das informações. Ao final, os dados tratados serão exportados para um arquivo CSV, disponibilizando seu uso para futuras análises e visualizações.

# BIBLIOTECAS

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# EXTRAÇÃO DOS DADOS

Primeiramente, vamos criar uma função que realiza web scraping no site do IBGE para coletar indicadores de estados brasileiros.

In [3]:
def info_por_estado(estado: str) -> dict:

    url = f'https://www.ibge.gov.br/cidades-e-estados/{estado}.html'

    req = requests.get(url)

    soup = BeautifulSoup(req.content, 'html.parser')

    indicadores = soup.select('.indicador')

    dict_estados = {
        ind.select('.ind-label')[0].text: ind.select('.ind-value')[0].text
        for ind in indicadores
    }

    dict_estados['Estado'] = estado

    return dict_estados

Agora, vamos criar uma lista contendo as siglas dos 27 estados brasileiros. Essa lista será usada para passar os estados como argumento para a função de scraping info_por_estado.


In [4]:
estados = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']

df_estados = [info_por_estado(estado) for estado in estados]

Para cada estado na lista estados, a função info_por_estado é executada.
Ela faz o scraping da página do estado no site do IBGE e retorna um dicionário com os dados.

Por fim, vamos transformar a lista de dicionários (df_estados) em um dataframe do Pandas.

In [5]:
df = pd.DataFrame(df_estados)

# BASE DE DADOS

Vamos examinar a base de dados para identificar possíveis problemas.

In [6]:
df.head()

,Governador,Capital,Gentílico,Área Territorial,População residente,Densidade demográfica,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Total de receitas brutas realizadas,Total de despesas brutas empenhadas,Rendimento mensal domiciliar per capita,Total de veículos,Estado
0,GLADSON DE LIMA CAMELI [2023],Rio Branco [2010],acriano,"164.173,429 km² [2022]",830.018 pessoas [2022],"5,06 hab/km² [2022]",147.350 matrículas [2023],"0,71 [2021]","11.137.410.342,21 R$ [2023]","10.302.403.420,85 R$ [2023]",1.095 R$ [2023],350.273 veículos [2023],AC
1,PAULO SURUAGY DO AMARAL DANTAS [2023],Maceió [2010],alagoano,"27.830,661 km² [2022]",3.127.683 pessoas [2022],"112,38 hab/km² [2022]",439.014 matrículas [2023],"0,684 [2021]","20.927.817.222,51 R$ [2023]","17.963.587.096,92 R$ [2023]",1.110 R$ [2023],1.095.144 veículos [2023],AL
2,CLÉCIO LUÍS VILHENA VIEIRA [2023],Macapá [2010],amapaense,"142.470,762 km² [2022]",733.759 pessoas [2022],"5,15 hab/km² [2022]",131.948 matrículas [2023],"0,688 [2021]","11.249.572.238,02 R$ [2023]","8.505.172.844,49 R$ [2023]",1.520 R$ [2023],242.574 veículos [2023],AP
3,WILSON MIRANDA LIMA [2023],Manaus [2010],amazonense,"1.559.255,881 km² [2022]",3.941.613 pessoas [2022],"2,53 hab/km² [2022]",692.623 matrículas [2023],"0,7 [2021]","33.030.751.885,07 R$ [2023]","30.029.370.828,75 R$ [2023]",1.172 R$ [2023],1.130.055 veículos [2023],AM
4,JERÔNIMO RODRIGUES SOUZA [2023],Salvador [2010],baiano,"564.760,429 km² [2022]",14.141.626 pessoas [2022],"25,04 hab/km² [2022]",1.865.574 matrículas [2023],"0,691 [2021]","80.290.524.858,16 R$ [2023]","77.868.037.416,52 R$ [2023]",1.139 R$ [2023],5.120.353 veículos [2023],BA


Podemos observar que os dados apresentados na tabela contêm elementos que podem ser considerados "ruído" ou "poluição de dados", como símbolos (km², RS, [2022], [2023], etc.), além de diferentes formatos para os números (como pontos decimais e separadores de milhares).

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 13 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Governador                               27 non-null     object
 1   Capital                                  27 non-null     object
 2   Gentílico                                27 non-null     object
 3   Área Territorial                         27 non-null     object
 4   População residente                      27 non-null     object
 5   Densidade demográfica                    27 non-null     object
 6   Matrículas no ensino fundamental        27 non-null     object
 7   IDH Índice de desenvolvimento humano    27 non-null     object
 8   Total de receitas brutas realizadas      27 non-null     object
 9   Total de despesas brutas empenhadas      27 non-null     object
 10  Rendimento mensal domiciliar per capita  27 non-null     object


Aqui, podemos observar que todas as colunas estão com o tipo de dado 'object', embora algumas delas representem valores numéricos. Isso indica os caracteres não numéricos (como símbolos, separadores de milhar, ou unidades) misturados nos dados estão fazendo com que o Pandas interprete as colunas como strings.

# LIMPEZA E TRATAMENTO DOS DADOS

Neste momento, vamos realizar a limpeza de caracteres indesejados no dataframe, removendo símbolos que estão "poluindo" os dados.

In [8]:
df = df.replace({
    '\.': '',
    ',': '.',
    '\[\d+\]': '',
    'hab/km²': '',
    'km²': '',
    'pessoas': '',
    'matrículas': '',
    'R\$.*': '',
    'veículos': ''
}, regex=True)

In [9]:
df.head()

,Governador,Capital,Gentílico,Área Territorial,População residente,Densidade demográfica,Matrículas no ensino fundamental,IDH Índice de desenvolvimento humano,Total de receitas brutas realizadas,Total de despesas brutas empenhadas,Rendimento mensal domiciliar per capita,Total de veículos,Estado
0,GLADSON DE LIMA CAMELI,Rio Branco,acriano,164173.429,830018,5.06,147350,0.71,11137410342.21,10302403420.85,1095,350273,AC
1,PAULO SURUAGY DO AMARAL DANTAS,Maceió,alagoano,27830.661,3127683,112.38,439014,0.684,20927817222.51,17963587096.92,1110,1095144,AL
2,CLÉCIO LUÍS VILHENA VIEIRA,Macapá,amapaense,142470.762,733759,5.15,131948,0.688,11249572238.02,8505172844.49,1520,242574,AP
3,WILSON MIRANDA LIMA,Manaus,amazonense,1559255.881,3941613,2.53,692623,0.7,33030751885.07,30029370828.75,1172,1130055,AM
4,JERÔNIMO RODRIGUES SOUZA,Salvador,baiano,564760.429,14141626,25.04,1865574,0.691,80290524858.16,77868037416.52,1139,5120353,BA


Após realizar a limpeza dos dados, vamos agora corrigir os tipos das colunas numéricas que precisam ser ajustadas.

In [10]:
colunas_numericas = ['Área Territorial', 'População residente', 'Densidade demográfica', 'Matrículas no ensino fundamental', 'IDH Índice de desenvolvimento humano', 'Total de receitas brutas realizadas', 'Total de despesas brutas empenhadas', 'Rendimento mensal domiciliar per capita', 'Total de veículos']

df[colunas_numericas] = df[colunas_numericas].apply(lambda x: x.str.strip())

df[colunas_numericas] = df[colunas_numericas].apply(pd.to_numeric)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 13 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Governador                               27 non-null     object 
 1   Capital                                  27 non-null     object 
 2   Gentílico                                27 non-null     object 
 3   Área Territorial                         27 non-null     float64
 4   População residente                      27 non-null     int64  
 5   Densidade demográfica                    27 non-null     float64
 6   Matrículas no ensino fundamental        27 non-null     int64  
 7   IDH Índice de desenvolvimento humano    27 non-null     float64
 8   Total de receitas brutas realizadas      27 non-null     float64
 9   Total de despesas brutas empenhadas      27 non-null     float64
 10  Rendimento mensal domiciliar per capita  27 non-null

Enfim, todas as correções foram realizadas, e o dataframe finalmente está em um formato apropriado para ser exportado para o formato csv.

# EXPORTANDO ARQUIVO CSV

In [12]:
df.to_csv('df.csv', index=False)